In [27]:
import importlib

import Wrapper.src.wrapper_api.wrappers.communication_wrapper as cw
import Wrapper.src.wrapper_api.models.active_communication as ac

importlib.reload(ac)
importlib.reload(cw)

from Wrapper.src.wrapper_api.models.active_communication import ActiveCommunication
from Wrapper.src.wrapper_api.wrappers.communication_wrapper import CommunicationWrapper


In [28]:
from pettingzoo.utils import AECEnv
from gymnasium import spaces
import numpy as np

class DummyEnv(AECEnv):
    def __init__(self):
        self.possible_agents = ["agent_0", "agent_1", "agent_2"]
        self.agent = self.possible_agents[2]

    def observe(self, agent):
        return {
            "self": np.array([1.0, 1.0]),
            "agent_0": np.array([0.5, 0.5]),
            "agent_1": np.array([0.8, 0.3])
        }
    def last(self, observe=True):
        return self.observe(self.agent_selection), 0.0, False, False, {}

    def observation_space(self, agent):
        return spaces.Dict({
            "self": spaces.Box(low=0, high=1.0, shape=(2,), dtype=np.float32),
            "agent_0": spaces.Box(low=0, high=1.0, shape=(2,), dtype=np.float32),
            "agent_1": spaces.Box(low=0, high=1.0, shape=(2,), dtype=np.float32),
        })
    def action_space(self, agent):
        return spaces.Discrete(3)

    def reset(self, seed=None, options=None): pass
    def step(self, action): pass

In [29]:
env = DummyEnv()
wrapper = CommunicationWrapper(env, failure_models=[])

# Override comms matrix manually
wrapper.comms_matrix = ActiveCommunication(agent_ids=env.possible_agents)

# Block agent 0 -> agent 2
wrapper.comms_matrix.update("agent_0", "agent_2", False)

# allow agent1 -> agent 2
wrapper.comms_matrix.update("agent_1", "agent_2", True)

In [30]:
obs = wrapper.observe("agent_2")
print("Filtered Observations (observe) for agent 2:")
for k, v in obs.items():
    print(f"{k}: {v}")

Filtered Observations (observe) for agent 2:
self: [1. 1.]
agent_0: [0. 0.]
agent_1: [0.8 0.3]


In [31]:
wrapper.env.agent_selection = "agent_2"
obs, reward, termination, truncation, info = wrapper.last(observe=True)

print("\nFiltered Observations (observe) for agent 2:")
for k, v in obs.items():
    print(f"{k}: {v}")


Filtered Observations (observe) for agent 2:
self: [1. 1.]
agent_0: [0. 0.]
agent_1: [0.8 0.3]


In [32]:
wrapper.env.agent_selection = "agent_0"

# Block agent 0 from sending to anyone
for receiver in env.possible_agents:
    if receiver != "agent_0":
        wrapper.comms_matrix.update("agent_0", receiver, False)

action = 2
filtered = wrapper.filter_action("agent_0", action)
print("\nFiltered action for agent 0:", filtered)


Filtered action for agent 0: 0
